In [0]:
CREATE OR REPLACE STREAMING TABLE customers_silver (
  alias STRING,
  customer_id BIGINT,
  customer_notes STRING,
  email STRING MASK ${default_catalog}.${admin_schema}.email_mask,
  email2 STRING MASK ${default_catalog}.${admin_schema}.email_mask,
  ip_address STRING MASK ${default_catalog}.${admin_schema}.ip_mask,
  md5_payment_instrument STRING,
  memo STRING,
  created_ts TIMESTAMP,
  modified_ts TIMESTAMP,
  name STRING,
  payment_instrument STRING,
  payment_instrument_type STRING,
  CONSTRAINT valid_email EXPECT (email LIKE '%@%.%'),
  CONSTRAINT valid_email2 EXPECT (email2 LIKE '%@%.%'),
  CONSTRAINT alias_not_null EXPECT (alias IS NOT NULL),
  CONSTRAINT name_not_null EXPECT (name IS NOT NULL)
)
COMMENT "Cleanesed customers from customers_bronze."
TBLPROPERTIES("quality" = "bronze")
-- WITH ROW FILTER usertraining.shared.payment_instrument_type_filter ON(payment_instrument_type)
AS
  SELECT
    alias,
    created_ts,
    customer_id,
    customer_notes,
    email,
    email2,
    ip_address,
    md5_payment_instrument,
    memo,
    modified_ts,
    name,
    payment_instrument,
    payment_instrument_type
  FROM stream(LIVE.customers_bronze);

In [0]:
CREATE OR REFRESH TEMPORARY STREAMING TABLE customers_silver_quarantine(
  CONSTRAINT quarantined_row EXPECT (alias IS NOT NULL OR name IS NOT NULL OR email LIKE '%@%.%' OR email2 LIKE '%@%.%')
)
PARTITIONED BY (is_quarantined)
AS
  SELECT
    *,
    NOT ((alias IS NOT NULL) and (name IS NOT NULL) and (email LIKE '%@%.%') and (email2 LIKE '%@%.%')) as is_quarantined
  FROM STREAM(customers_silver);

CREATE TEMPORARY LIVE VIEW valid_customers_silver AS
SELECT * FROM customers_silver_quarantine WHERE is_quarantined=FALSE;

CREATE TEMPORARY LIVE VIEW invalid_customers_silver AS
SELECT * FROM customers_silver_quarantine WHERE is_quarantined=TRUE;